In [ ]:
import os
import numpy as np
import scipy.special
from cosmoprimo import Cosmology, PowerSpectrumBAOFilter
import argparse
import multiprocessing
from utils import savedir_template, cosmology, redshift_distributions

# # Parse command-line arguments
# parser = argparse.ArgumentParser()
# parser.add_argument('--args.cosmology_template', help='mice, planck or lognormal_Y6BAO', default='mice', type=str)
# parser.add_argument('--args.include_wiggles', help='y or n', default='y', type=str)
# args = parser.parse_args()

# args.cosmology_template = args.cosmology_template
# args.include_wiggles = '' if args.include_wiggles == 'y' else '_noBAO'

# Arguments
class Args:
    def __init__(self):
        self.cosmology_template = "planck"
        self.nz_flag = "fid"
        self.include_wiggles = "y"
args = Args()
args.include_wiggles = '' if args.include_wiggles == 'y' else '_noBAO'

# 0. Path for saving the template
savedir = savedir_template(include_wiggles=args.include_wiggles, nz_flag=args.nz_flag, cosmology_template=args.cosmology_template)()
os.makedirs(savedir, exist_ok=True)

# 1. Galaxy bias. I give it random values, since we save the three components of w(theta) separately. However, we use it for checking the results
galaxy_bias = {0: 1.3, 1: 1.9, 2: 4, 3: 2.1, 4: 3, 5: 0.7}

# 2. Numerical resolution of the calculation

# Nz, Nk, Nmu, Nr = 10**3, 2*10**5, 5*10**4, 5*10**4
Nz, Nk, Nmu, Nr = 10**3, 10**4, 10**4, 10**4

mu_vector = np.linspace(-1, 1, Nmu)
legendre = {ell: scipy.special.eval_legendre(ell, mu_vector) for ell in [0, 2, 4]}
r_12_vector = 10**np.linspace(np.log10(10**-2), np.log10(10**5), Nr)

# 3. Values of theta
theta = 10**np.linspace(np.log10(0.001), np.log10(179.5), 10**2) * np.pi/180

# 4. Redshift distributions
nz_instance = redshift_distributions(args.nz_flag)

# 5. Cosmology
params = cosmology(args.cosmology_template)
cosmo = Cosmology(
    h=params.h,
    Omega_cdm=params.Omega_m - params.Omega_b - params.Omega_nu_massive,
    Omega_b=params.Omega_b,
    sigma8=params.sigma_8,
    n_s=params.n_s,
    Omega_ncdm=params.Omega_nu_massive,
    N_eff=3.046,
    engine='class'
)

# 6. Power spectrum. We use the implementation in cosmoprimo
pkz = cosmo.get_fourier(engine='class').pk_interpolator()
pk = pkz.to_1d(z=0)
kh = 10 ** np.linspace(np.log10(1e-4 / cosmo.h), np.log10(1e2), Nk)
Pk_wigg = pk(kh) / cosmo.h**3

pknow = PowerSpectrumBAOFilter(pk, engine='wallish2018').smooth_pk_interpolator()
Pk_nowigg = pknow(kh) / cosmo.h**3

k = kh * cosmo.h
np.savetxt(f'{savedir}/Pk_full.txt', np.column_stack([k, Pk_wigg, Pk_nowigg]))

k_s = 0.2 * cosmo.h
ell_BAO = 110 / cosmo.h

q = 10 ** np.linspace(np.log10(k.min()), np.log10(k_s), Nk)
Pq_nowigg = np.interp(q, k, Pk_nowigg)

x = q * ell_BAO
j_0 = np.sin(x) / x
j_2 = (3 / x**2 - 1) * j_0 - 3 * np.cos(x) / x**2

Sigma_0 = np.sqrt(np.trapz(Pq_nowigg * (1 - j_0 + 2 * j_2), q) / (6 * np.pi**2))
delta_Sigma_0 = np.sqrt(np.trapz(Pq_nowigg * j_2, q) / (2 * np.pi**2))

print([Sigma_0 * cosmo.h, delta_Sigma_0 * cosmo.h])

# Calculation of xi_ell(s)

def compute_xi_ell(bin_z):
    
    z = nz_instance.z_average(bin_z)
    z_array = nz_instance.z_vector(bin_z)

    D = cosmo.growth_factor(z)
    f = cosmo.growth_rate(z)

    Sigma = D * Sigma_0
    delta_Sigma = D * delta_Sigma_0

    Sigma_paral = (1 + f) * Sigma
    Sigma_perp = Sigma
    Sigma_tot_vector = np.sqrt(
        mu_vector**2 * Sigma_paral**2 
        + (1 - mu_vector**2) * Sigma_perp**2 
        + f * mu_vector**2 * (mu_vector**2 - 1) * delta_Sigma**2
    )

    print(f"{bin_z} - Computing Pk_ell...")

    def compute_pk_multipoles(k, Pk_wigg, Pk_nowigg):
        if args.include_wiggles == '':
            pk_term = (Pk_wigg - Pk_nowigg) * np.exp(-k**2 * Sigma_tot_vector**2) + Pk_nowigg
        elif args.include_wiggles == '_noBAO':
            pk_term = Pk_nowigg

        pk_dict = {}
        for ell in [0, 2, 4]:
            leg = legendre[ell]
            coeff = (2 * ell + 1) / 2

            pk_dict[f'Pk_{ell}'] = coeff * np.trapz((galaxy_bias[bin_z] + mu_vector**2 * f)**2 * pk_term * leg, mu_vector)
            pk_dict[f'Pk_{ell}_bb'] = coeff * np.trapz(pk_term * leg, mu_vector)
            pk_dict[f'Pk_{ell}_bf'] = coeff * np.trapz(2 * mu_vector**2 * f * pk_term * leg, mu_vector)
            pk_dict[f'Pk_{ell}_ff'] = coeff * np.trapz(mu_vector**4 * f**2 * pk_term * leg, mu_vector)

        return pk_dict

    pk_ell_dict = {key: np.zeros(len(k)) for key in [
        'Pk_0', 'Pk_2', 'Pk_4',
        'Pk_0_bb', 'Pk_2_bb', 'Pk_4_bb',
        'Pk_0_bf', 'Pk_2_bf', 'Pk_4_bf',
        'Pk_0_ff', 'Pk_2_ff', 'Pk_4_ff'
    ]}

    for i, k_val in enumerate(k):
        pk_dict = compute_pk_multipoles(
            k_val, Pk_wigg[i], Pk_nowigg[i]
        )
        for key, value in pk_dict.items():
            pk_ell_dict[key][i] = value

    np.savetxt(
        savedir + f'/Pk_ell_bb_bin{bin_z}.txt', 
        np.column_stack([k, pk_ell_dict['Pk_0_bb'], pk_ell_dict['Pk_2_bb'], pk_ell_dict['Pk_4_bb']])
    )
    np.savetxt(
        savedir + f'/Pk_ell_bf_bin{bin_z}.txt', 
        np.column_stack([k, pk_ell_dict['Pk_0_bf'], pk_ell_dict['Pk_2_bf'], pk_ell_dict['Pk_4_bf']])
    )
    np.savetxt(
        savedir + f'/Pk_ell_ff_bin{bin_z}.txt', 
        np.column_stack([k, pk_ell_dict['Pk_0_ff'], pk_ell_dict['Pk_2_ff'], pk_ell_dict['Pk_4_ff']])
    )

    diff_check = [
        abs(pk_ell_dict['Pk_0'] - pk_ell_dict['Pk_0_bb'] * galaxy_bias[bin_z]**2 - pk_ell_dict['Pk_0_bf'] * galaxy_bias[bin_z] - pk_ell_dict['Pk_0_ff']).max(),
        abs(pk_ell_dict['Pk_2'] - pk_ell_dict['Pk_2_bb'] * galaxy_bias[bin_z]**2 - pk_ell_dict['Pk_2_bf'] * galaxy_bias[bin_z] - pk_ell_dict['Pk_2_ff']).max(),
        abs(pk_ell_dict['Pk_4'] - pk_ell_dict['Pk_4_bb'] * galaxy_bias[bin_z]**2 - pk_ell_dict['Pk_4_bf'] * galaxy_bias[bin_z] - pk_ell_dict['Pk_4_ff']).max(),
    ]

    print(f"These should be 0: {diff_check}")
    print(f"{bin_z} - Pk_ell computed!")
    print()
    
    print(f"{bin_z} - Computing xi_ell...")

    def compute_xi_multipoles(r):
        x = k * r
        x_square_inv = 1 / x**2
        sinc_x = np.sin(x) / x
        cos_x_redef = np.cos(x) * x_square_inv

        j_0_x = sinc_x
        j_2_x = (3 * x_square_inv - 1) * sinc_x - 3 * cos_x_redef
        j_4_x = 5 * (2 - 21 * x_square_inv) * cos_x_redef + (1 - 45 * x_square_inv + 105 * x_square_inv**2) * sinc_x

        xi_dict = {}
        for ell, j_ell_x in zip([0, 2, 4], [j_0_x, j_2_x, j_4_x]):
            xi_dict[f'xi_{ell}'] = np.trapz(j_ell_x * k**2 / (2 * np.pi**2) * pk_ell_dict[f'Pk_{ell}'], k)
            xi_dict[f'xi_{ell}_bb'] = np.trapz(j_ell_x * k**2 / (2 * np.pi**2) * pk_ell_dict[f'Pk_{ell}_bb'], k)
            xi_dict[f'xi_{ell}_bf'] = np.trapz(j_ell_x * k**2 / (2 * np.pi**2) * pk_ell_dict[f'Pk_{ell}_bf'], k)
            xi_dict[f'xi_{ell}_ff'] = np.trapz(j_ell_x * k**2 / (2 * np.pi**2) * pk_ell_dict[f'Pk_{ell}_ff'], k)

        return xi_dict

    xi_ell_dict = {key: np.zeros(Nr) for key in [
        'xi_0', 'xi_2', 'xi_4',
        'xi_0_bb', 'xi_2_bb', 'xi_4_bb',
        'xi_0_bf', 'xi_2_bf', 'xi_4_bf',
        'xi_0_ff', 'xi_2_ff', 'xi_4_ff'
    ]}

    for i, r_12 in enumerate(r_12_vector):
        xi_dict = compute_xi_multipoles(r_12)
        for key, value in xi_dict.items():
            xi_ell_dict[key][i] = value

        if (i + 1) % (Nr // 10) == 0:
            print(f"{bin_z} - {int((i + 1) / Nr * 100)}%")

    np.savetxt(
        savedir + f'/xi_ell_bb_bin{bin_z}.txt',
        np.column_stack([r_12_vector, xi_ell_dict['xi_0_bb'], xi_ell_dict['xi_2_bb'], xi_ell_dict['xi_4_bb']])
    )
    np.savetxt(
        savedir + f'/xi_ell_bf_bin{bin_z}.txt',
        np.column_stack([r_12_vector, xi_ell_dict['xi_0_bf'], xi_ell_dict['xi_2_bf'], xi_ell_dict['xi_4_bf']])
    )
    np.savetxt(
        savedir + f'/xi_ell_ff_bin{bin_z}.txt',
        np.column_stack([r_12_vector, xi_ell_dict['xi_0_ff'], xi_ell_dict['xi_2_ff'], xi_ell_dict['xi_4_ff']])
    )

    diff_check = [
        abs(xi_ell_dict['xi_0'] - xi_ell_dict['xi_0_bb'] * galaxy_bias[bin_z]**2 - xi_ell_dict['xi_0_bf'] * galaxy_bias[bin_z] - xi_ell_dict['xi_0_ff']).max(),
        abs(xi_ell_dict['xi_2'] - xi_ell_dict['xi_2_bb'] * galaxy_bias[bin_z]**2 - xi_ell_dict['xi_2_bf'] * galaxy_bias[bin_z] - xi_ell_dict['xi_2_ff']).max(),
        abs(xi_ell_dict['xi_4'] - xi_ell_dict['xi_4_bb'] * galaxy_bias[bin_z]**2 - xi_ell_dict['xi_4_bf'] * galaxy_bias[bin_z] - xi_ell_dict['xi_4_ff']).max(),
    ]

    print(f"These should be 0: {diff_check}")
    print(f"{bin_z} - xi_ell computed!")
    
    return xi_ell_dict

pool = multiprocessing.Pool(nz_instance.nbins)
xi_ell_dict = pool.map(compute_xi_ell, range(nz_instance.nbins))

pool.close()
pool.join()

# Calculation of w(theta)

n_cpu = multiprocessing.cpu_count()

for bin_z in range(nz_instance.nbins):

    z_values = nz_instance.z_vector(bin_z, Nz=Nz)
    D_values = cosmo.growth_factor(z_values)
    phi_values = nz_instance.nz_interp(z_values, bin_z) * D_values
    r_values = cosmo.comoving_radial_distance(z_values)/cosmo.h

    integrand_results = {key: np.zeros((len(z_values), len(z_values))) for key in [
        'integrand', 'integrand_bb', 'integrand_bf', 'integrand_ff'
    ]}
    
    print(f"{bin_z} - Computing wtheta...")

    def wtheta_calculator(theta):
        for i in np.arange(0, len(z_values)):
            for j in np.arange(0, len(z_values)):
                if j < i:
                    for key in integrand_results.keys():
                        integrand_results[key][i, j] = integrand_results[key][j, i]
                else:
                    r_12 = np.sqrt(r_values[i]**2 + r_values[j]**2 - 2 * r_values[i] * r_values[j] * np.cos(theta))
                    mu = (r_values[j] - r_values[i]) / r_12
                    try:
                        integrand_results['integrand'][i, j] = phi_values[i] * phi_values[j] * sum([
                            np.interp(r_12, r_12_vector, xi_ell_dict[bin_z]['xi_0']) * np.interp(mu, mu_vector, legendre[0]),
                            np.interp(r_12, r_12_vector, xi_ell_dict[bin_z]['xi_2']) * np.interp(mu, mu_vector, legendre[2]),
                            np.interp(r_12, r_12_vector, xi_ell_dict[bin_z]['xi_4']) * np.interp(mu, mu_vector, legendre[4])
                        ])
                        integrand_results['integrand_bb'][i, j] = phi_values[i] * phi_values[j] * sum([
                            np.interp(r_12, r_12_vector, xi_ell_dict[bin_z]['xi_0_bb']) * np.interp(mu, mu_vector, legendre[0]),
                            np.interp(r_12, r_12_vector, xi_ell_dict[bin_z]['xi_2_bb']) * np.interp(mu, mu_vector, legendre[2]),
                            np.interp(r_12, r_12_vector, xi_ell_dict[bin_z]['xi_4_bb']) * np.interp(mu, mu_vector, legendre[4])
                        ])
                        integrand_results['integrand_bf'][i, j] = phi_values[i] * phi_values[j] * sum([
                            np.interp(r_12, r_12_vector, xi_ell_dict[bin_z]['xi_0_bf']) * np.interp(mu, mu_vector, legendre[0]),
                            np.interp(r_12, r_12_vector, xi_ell_dict[bin_z]['xi_2_bf']) * np.interp(mu, mu_vector, legendre[2]),
                            np.interp(r_12, r_12_vector, xi_ell_dict[bin_z]['xi_4_bf']) * np.interp(mu, mu_vector, legendre[4])
                        ])
                        integrand_results['integrand_ff'][i, j] = phi_values[i] * phi_values[j] * sum([
                            np.interp(r_12, r_12_vector, xi_ell_dict[bin_z]['xi_0_ff']) * np.interp(mu, mu_vector, legendre[0]),
                            np.interp(r_12, r_12_vector, xi_ell_dict[bin_z]['xi_2_ff']) * np.interp(mu, mu_vector, legendre[2]),
                            np.interp(r_12, r_12_vector, xi_ell_dict[bin_z]['xi_4_ff']) * np.interp(mu, mu_vector, legendre[4])
                        ])
                    except ValueError:
                        print([r_12, mu])

        w_dict = {}
        w_dict['wtheta'] = np.trapz(np.trapz(integrand_results['integrand'], z_values), z_values)
        w_dict['wtheta_bb'] = np.trapz(np.trapz(integrand_results['integrand_bb'], z_values), z_values)
        w_dict['wtheta_bf'] = np.trapz(np.trapz(integrand_results['integrand_bf'], z_values), z_values)
        w_dict['wtheta_ff'] = np.trapz(np.trapz(integrand_results['integrand_ff'], z_values), z_values)

        return w_dict
    
    pool = multiprocessing.Pool(n_cpu)
    w_dict = pool.map(wtheta_calculator, theta)

    pool.close()
    pool.join()
    
    wtheta_dict = {'wtheta': np.array([w_dict[i]['wtheta'] for i in range(len(theta))]),
                   'wtheta_bb': np.array([w_dict[i]['wtheta_bb'] for i in range(len(theta))]),
                   'wtheta_bf': np.array([w_dict[i]['wtheta_bf'] for i in range(len(theta))]),
                   'wtheta_ff': np.array([w_dict[i]['wtheta_ff'] for i in range(len(theta))])
                  }

    np.savetxt(
        savedir + f'/wtheta_bb_bin{bin_z}.txt',
        np.column_stack([theta, wtheta_dict['wtheta_bb'], wtheta_dict['wtheta_bb'], wtheta_dict['wtheta_bb']])
    )
    np.savetxt(
        savedir + f'/wtheta_bf_bin{bin_z}.txt',
        np.column_stack([theta, wtheta_dict['wtheta_bf'], wtheta_dict['wtheta_bf'], wtheta_dict['wtheta_bf']])
    )
    np.savetxt(
        savedir + f'/wtheta_ff_bin{bin_z}.txt',
        np.column_stack([theta, wtheta_dict['wtheta_ff'], wtheta_dict['wtheta_ff'], wtheta_dict['wtheta_ff']])
    )

    diff_check = [
        abs(wtheta_dict['wtheta'] - wtheta_dict['wtheta_bb'] * galaxy_bias[bin_z]**2 - wtheta_dict['wtheta_bf'] * galaxy_bias[bin_z] - wtheta_dict['wtheta_ff']).max(),
    ]

    print(f"This should be 0: {diff_check}")
    print(f"{bin_z} - wtheta computed!")


In [ ]:
Args()

In [ ]:
args